In [2]:
# default_exp metrics

In [3]:
#export
import torch.nn.utils as nn_utils
from fastai.torch_basics import *
from fastai.data.all import *
from fastai.basics import *
from fastai.metrics import *
from dataclasses import field,asdict
from typing import List,Any,Dict,Callable
from collections import deque
import gym
from torch.optim import *

from fastrl.data import *
from fastrl.async_data import *
from fastrl.basic_agents import *
from fastrl.learner import *
from fastai.callback.progress import *

import ptan

if IN_NOTEBOOK:
    from IPython import display
    import PIL.Image

# RL Metrics

> Metrics for tracking the progress of reinforcement learning agents.

In [4]:
# export
class AvgEpisodeRewardMetric(Metric):
    def __init__(self):self.rolling_rewards=deque([0],maxlen=100)
        
    def accumulate(self,learn):
#         yb=learn.yb
#         print(len(yb),len(yb[0]),yb)
        yb=[]
        for i in range(len(learn.xb[0])):
#             print(learn.yb)
            yb.append(ExperienceFirstLast(learn.xb[0][i].cpu().detach(),*(learn.yb[j][i].cpu().detach() for j in range(len(learn.yb)))))
#             print(yb[-1])
#         yb=[ExperienceFirstLast([0],*(yb[k][i] for k in range(len(yb)))) for i in range(len(yb[0]))]
#         yb=[for yb.items()]
#         print([o.done for o in yb if o.done])
        if len([float(o.episode_reward) for o in yb if o.done and int(o.episode_reward)!=0])==0:return
#         print([o.episode_reward for o in yb if o.done])
        r=np.average([float(o.episode_reward) for o in yb if o.done and int(o.episode_reward)!=0])
#         print([y for y in yb if y.absolute_end])
#         for r in rewards:
        if r!=0:self.rolling_rewards.extend([r])
#         print(len(rewards))
#         if len(rewards)!=0:self.r=sum(rewards)/len(rewards)
        
    @property
    def value(self):return np.mean(self.rolling_rewards) if len(self.rolling_rewards)!=1 else self.rolling_rewards[0]
    @property
    def name(self):return 'avg_episode_r'

In [5]:
from fastrl.actorcritic.a3c_data import *

In [6]:
env='CartPole-v1'

block=AsyncExperienceBlock(
    experience_block=partial(FirstLastExperienceBlock,a=0,seed=0,dls_kwargs={'bs':1,'n_steps':4,'num_workers':0,
                                                                             'verbose':False,'indexed':True,'shuffle_train':False}),
    n_processes=1,
    n=128
)
blk=IterableDataBlock(blocks=(block),
                      splitter=FuncSplitter(lambda x:False),
#                       batch_tfms=lambda x:(x[0],x),
                     )
dls=blk.dataloaders([env]*1,bs=3)

model=LinearA2C((4,),2)
agent=ActorCriticAgent(model=model)
learner=A3CLearner(dls,agent=agent,cbs=[A3CTrainer],reward_steps=4,
                   metrics=[AvgEpisodeRewardMetric()])
learner.fit(3)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [7]:
# hide
from nbdev.export2html import *
from nbdev.export import *
notebook2script()
notebook2html()

Converted 00_core.ipynb.
Converted 01_wrappers.ipynb.
Converted 03_basic_agents.ipynb.
Converted 04_learner.ipynb.
Converted 05a_data.ipynb.
Converted 05b_async_data.ipynb.
Converted 06_basic_train.ipynb.
Converted 13_metrics.ipynb.
Converted 14_actorcritic.sac.ipynb.
Converted 15_actorcritic.a3c_data.ipynb.
Converted 16_actorcritic.a2c.ipynb.
Converted index.ipynb.


converting: /opt/project/fastrl/nbs/16_actorcritic.a2c.ipynb
converting: /opt/project/fastrl/nbs/03_basic_agents.ipynb
converting: /opt/project/fastrl/nbs/13_metrics.ipynb
